In [1]:
import pandas as pd
from custom_func import *

In [24]:
df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx")
df_P02_007 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007.xlsx")

In [25]:
colnames = {
    "Область": "region",
    "Податки на одну особу (p2_01)":'p2_01_raw',
    "Дохід без міжбюдж. трансфертів (p2_02)":'p2_02_raw',
    "Дохід без міжбюдж. трансфертів порівняно з минулим періодом (p2_03)": "p2_03_raw",
    "Капітальні видатки (p2_04)":'p2_04_raw',
    "Плата за землю (p2_05)":'p2_05_raw'
}

df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx").rename(columns=colnames)

In [26]:
normalize_parameter(df_P02_006, "p2_01_raw", "p2_01")
normalize_parameter(df_P02_006, "p2_02_raw", "p2_02")
normalize_parameter(df_P02_006, "p2_03_raw", "p2_03")
normalize_parameter(df_P02_006, "p2_04_raw", "p2_04")
normalize_parameter(df_P02_006, "p2_05_raw", "p2_05")

df_P02_006_fin = df_P02_006.loc[:, df_P02_006.columns.str.contains("region|^p")]

Емпіричні границі: (273.1319492833434, 1419.510256667407)
Коефіцієнти нормалізації: (0.0008723123889895594, -0.23825638318872844)
Параметр p2_01 нормалізовано і додано до таблиці

Емпіричні границі: (689342314.3199999, 6955395883.62)
Коефіцієнти нормалізації: (1.595900815306488e-10, -0.11001219614485493)
Параметр p2_02 нормалізовано і додано до таблиці

Емпіричні границі: (0.624658426803403, 1.168221845737069)
Коефіцієнти нормалізації: (1.8397117340268172, -1.1491914375489523)
Параметр p2_03 нормалізовано і додано до таблиці

Емпіричні границі: (64811628, 1646013700)
Коефіцієнти нормалізації: (6.324302362791237e-10, -0.04098883320967467)
Параметр p2_04 нормалізовано і додано до таблиці

Емпіричні границі: (60392282.4, 729124352.5)
Коефіцієнти нормалізації: (1.4953671951914363e-09, -0.09030863794369715)
Параметр p2_05 нормалізовано і додано до таблиці



---

# Розрахунки на даних, отриманих з prom.ua

In [28]:
df_P02_007_fin = df_P02_007.loc[df_P02_007["status"].eq("complete")].copy()

In [29]:
df_P02_007_fin["Чи відкриті торги?"] = df_P02_007_fin["procurementMethod"].eq("open").astype(int)
open_procur = df_P02_007_fin.loc[df_P02_007_fin['Чи відкриті торги?']==1].groupby('region')["tenderValue"].sum().rename('open')
all_procur = df_P02_007_fin.groupby('region')["tenderValue"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

In [30]:
df_P02_007_fin

,region,open,all,p2_06_raw
0,Івано-Франківська,4.063802e+08,4.063802e+08,1.0
1,Волинська,8.175949e+07,8.175949e+07,1.0
2,Вінницька,9.636960e+08,9.636960e+08,1.0
3,Дніпропетровська,6.501261e+08,6.501261e+08,1.0
4,Донецька,8.316324e+08,8.316324e+08,1.0
5,Житомирська,8.569403e+08,8.569403e+08,1.0
6,Закарпатська,3.791655e+08,3.791655e+08,1.0
7,Запорізька,7.947314e+08,7.947314e+08,1.0
8,Київська,6.763841e+08,6.763841e+08,1.0
9,Кіровоградська,3.896431e+08,3.896431e+08,1.0


---

# Розрахунки на даних bi.prozorro

In [8]:
df = pd.read_excel("./../v0.5/inputs/P2/P02_007_prozorro.xlsx")

In [9]:
df.shape

(84049, 19)

In [11]:
open_procedures = ["Відкриті торги", "Відкриті торги із публікацією англ. мовою"]
df["Чи відкриті торги?"] = df["Процедура закупівлі"].isin(open_procedures).astype(int)

df = df.loc[df["Статуси процедур"].eq("Завершена закупівля")].copy()
open_procur = df.loc[df['Чи відкриті торги?']==1].groupby('region')["Очікувана вартість"].sum().rename('open')
all_procur = df.groupby('region')["Очікувана вартість"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

normalize_parameter(df_P02_007_fin, "p2_06_raw", "p2_06", min_bound=0)

Емпіричні границі: (0, 0.8545996331625495)
Коефіцієнти нормалізації: (1.1701385785755358, -0.0)
Параметр p2_06 нормалізовано і додано до таблиці



In [24]:
df_P02_007_fin

,region,open,all,p2_06_raw,p2_06
0,Івано-Франківська,1.829942e+08,3.123609e+08,0.585842,0.685517
1,Волинська,7.647532e+07,1.607761e+08,0.475664,0.556592
2,Вінницька,4.021977e+08,4.896396e+08,0.821416,0.961170
3,Дніпропетровська,5.128624e+08,1.084083e+09,0.473084,0.553574
4,Донецька,6.864578e+08,9.179936e+08,0.747781,0.875007
5,Житомирська,5.066892e+08,5.928966e+08,0.854600,1.000000
6,Закарпатська,4.114011e+08,5.604555e+08,0.734048,0.858938
7,Запорізька,3.365341e+08,4.994853e+08,0.673762,0.788395
8,Київська,4.823933e+08,7.265544e+08,0.663947,0.776909
9,Кіровоградська,2.123291e+08,2.940909e+08,0.721985,0.844822
